# Tavily搜索

[Tavily](https://tavily.com/) 是专为AI代理（LLMs）构建的搜索引擎，能够快速提供实时、准确且基于事实的结果。Tavily提供了两个主要端点，其中之一是搜索功能，为LLMs和RAG提供了定制的搜索结果。

本指南简要介绍了如何快速开始使用Tavily [工具](/docs/integrations/tools/)。如需了解Tavily工具的完整信息，请参阅[API参考文档](https://v03.api.js.langchain.com/modules/_langchain_tavily.html)。

## 概览

### 集成详情

| 类别 | 包 | [Python支持](https://python.langchain.com/docs/integrations/tools/tavily_search/) | 最新包 |
| :--- | :--- | :---: | :---: |
| [TavilySearch](https://api.js.langchain.com/classes/_langchain_tavily.TavilySearch.html) | [`@langchain/tavily`](https://www.npmjs.com/package/@langchain/tavily) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/tavily?style=flat-square&label=%20&) |

## 安装配置

该集成功能位于 `@langchain/tavily` 包中，你可以通过以下方式安装：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/tavily @langchain/core
</Npm2Yarn>
```

### 凭证

请在这里 [这里](https://app.tavily.com) 设置API密钥，并将其设置为名为 `TAVILY_API_KEY` 的环境变量。

```typescript
process.env.TAVILY_API_KEY = "YOUR_API_KEY"
```

同时，为了获得最佳的可观测性，建议（非必须）设置 [LangSmith](https://smith.langchain.com/)：

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

你可以通过以下方式导入并实例化 `TavilySearch` 工具：

In [1]:
import { TavilySearch } from "@langchain/tavily";

const tool = new TavilySearch({
  maxResults: 5,
  topic: "general",
  // includeAnswer: false,
  // includeRawContent: false,
  // includeImages: false,
  // includeImageDescriptions: false,
  // searchDepth: "basic",
  // timeRange: "day",
  // includeDomains: [],
  // excludeDomains: [],
});

## 调用

### [使用参数直接调用](/docs/concepts/tools)

Tavily 搜索工具在调用时接受以下参数：

* `query`（必填）：一个自然语言搜索查询

* 调用期间还可以设置以下参数：`includeImages`、`searchDepth`、`timeRange`、`includeDomains`、`excludeDomains`、`includeImages`。

* 出于可靠性和性能考虑，某些影响响应大小的参数无法在调用期间修改：`includeAnswer` 和 `includeRawContent`。这些限制可防止出现意外的上下文窗口问题，并确保结果的一致性。

In [ ]:
await tool.invoke({
  query: "what is the current weather in SF?"
});

### [通过ToolCall调用](/docs/concepts/tools)

我们也可以使用模型生成的`ToolCall`来调用工具，在这种情况下将返回一个`ToolMessage`：

In [ ]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    query: "what is the current weather in SF?"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}

await tool.invoke(modelGeneratedToolCall)

## 链式调用

我们可以通过首先将其绑定到一个[工具调用模型](/docs/how_to/tool_calling/)，然后调用它，以实现工具的链式使用：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [7]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("what is the current weather in sf?");

In [ ]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

## 代理

关于如何在代理中使用LangChain工具的指南，请参阅 [LangGraph.js](https://langchain-ai.github.io/langgraphjs/how-tos/#tool-calling) 文档。

## API 参考文档

有关 Tavily Search API 所有功能和配置的详细文档，请访问 API 参考页面：

https://docs.tavily.com/documentation/api-reference/endpoint/search

## 相关内容

* 工具[概念指南](https://js.langchain.com/docs/concepts/tools/)

* 工具[操作指南](https://js.langchain.com/docs/how_to/#tools)